# Experiments with a custom Docker image not based on the netunicorn agent image

Sometimes you need to use the netUnicorn platform to just deploy your Docker containers to the required nodes, and your Docker containers are not based on the netUnicorn agent image. In this case, you still can use the netUnicorn platform to deploy your containers to the required nodes, and the platform will only deploy containers and wait for timeout or cancel command. The netUnicorn system would not be able to detect experiment completion, so timeout or cancellation are two only triggers to stop the experiment.

Let's start with typical preparations.

In [ ]:
import os

from netunicorn.base import DockerImage, Experiment, Pipeline
from netunicorn.client.remote import RemoteClient

In [ ]:
if '.env' in os.listdir():
    from dotenv import load_dotenv
    load_dotenv(".env")

endpoint = os.environ.get('NETUNICORN_ENDPOINT') or 'http://localhost:26611'
login = os.environ.get('NETUNICORN_LOGIN') or 'test'
password = os.environ.get('NETUNICORN_PASSWORD') or 'test'

In [ ]:
client = RemoteClient(endpoint=endpoint, login=login, password=password)
client.healthcheck()
nodes = client.get_nodes().take(5)

To implement the described scenario, you need to assign a dummy pipeline (e.g., empty one) to the required nodes, set timeout to the value you want the docker container to stay alive, and set your docker image name to the deployment environment definition. The following cell shows how to do it.

In [ ]:
pipeline = Pipeline()  # Dummy Pipeline
experiment = Experiment().map(pipeline, nodes)  # map to the required nodes
for deployment in experiment.deployments:
    deployment.keep_alive_timeout_minutes = 10**5  # set timeout to whatever time your experiment needs
    deployment.environment_definition = DockerImage(image="your_image_name:latest")  # set your image name

Afterward, you can proceed with the default experiment workflow.

In [ ]:
experiment_name = 'experiment_cool_name'
client.prepare_experiment(experiment, experiment_name)

In [ ]:
client.start_execution(experiment_name)

As a result, netUnicorn will deploy containers and will expect keep-alive pings from them. As the containers are not based on the agent's image and will not provide keep-alive pings, the netUnicorn system will wait for the timeout or cancellation command to stop the experiment.